In [85]:
import pymongo

In [86]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client['new_db']
collection = db['raw_news']

In [87]:
raw_data = list(collection.find())

In [88]:
len(raw_data)

227

In [89]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import spacy


In [90]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\AjayPatil\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\AjayPatil\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\AjayPatil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\AjayPatil\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [91]:
nlp = spacy.load("en_core_web_lg")

In [92]:
lemmatizer = WordNetLemmatizer()

In [93]:
def preprocess_news(news):
    text = f"{news['title']} {news}['desc] {' '.join(news['content'])}"
    
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    text = re.sub(r"\s+"," ", text).strip()
    
    text = text.lower()
    
    tokens = word_tokenize(text)
    
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    lemmatized_token = [lemmatizer.lemmatize(token) for token in tokens]
    return " ".join(lemmatized_token)

In [94]:
def extract_entities(text):
    doc = nlp(text)
    stock_names = [ent.text for ent in doc.ents if ent.label_ in {"ORG", "PRODUCT", "GPE"}]
    return list(set(stock_names))


In [95]:
processed_data = []

In [96]:
for news in raw_data:
    processed_text = preprocess_news(news)
    stock_names = extract_entities(news['title'] + " " + processed_text)
    
    processed_data.append({
        "_id": news["_id"],
        "processed_text": processed_text,
        "date": news["date"],
        "datetime": news["datetime"],
        "stock_name":stock_names,
        
    })

In [97]:
processed_data[1]

{'_id': ObjectId('676e8cbb49f67e6ab04c1ed7'),
 'processed_text': 'agri pick report december geojit financial service id objectidecbbfeabced title agri pick report december geojit financial service desc according geojit financial service national commodity derivative exchange monday said would live trading session feb account presentation union budget aprmar date december datetime december ist content geojit financial services report daily agri pick national commodity derivative exchange monday said would live trading session feb account presentation union budget aprmar farmer karnataka sown rabi crop million hectare saturday year million hectare sown corresponding period last year according report released state agriculture government chana acreage state rose nearly year million hectare hectare sown last year acreage horse gram kulthi dal hectare hectare last year report showed total area sown pulse million hectare million hectare year ago jowar wheat chana onion chilli key crop grown 

In [98]:
collection = db['processed_news']
x = collection.insert_many(processed_data)

BulkWriteError: batch op errors occurred, full error: {'writeErrors': [{'index': 0, 'code': 11000, 'errmsg': "E11000 duplicate key error collection: new_db.processed_news index: _id_ dup key: { _id: ObjectId('676e8cb149f67e6ab04c1ed6') }", 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('676e8cb149f67e6ab04c1ed6')}, 'op': {'_id': ObjectId('676e8cb149f67e6ab04c1ed6'), 'processed_text': 'mazagon dock share trade percent lower following stock split id objectidecbfeabced title mazagon dock share trade percent lower following stock split desc fall stock price come following exdate stock split today date december datetime december ist content share mazagon dock shipbuilder focus higher usual today given tradeex split scheduled december stock witnessed decline percent trading r pm come company gear stock splitxaunder stock split existing share mazagon dock face value r divided two new share face value r per observation medium report certain trading application could showing unadjusted 

In [68]:
# x.inserted_ids

In [69]:
client.close()